In [ ]:
from pathlib import Path
import os
import pandas as pd

ROOT = Path().resolve().parent
os.chdir(ROOT)

target = ROOT/"data"/"raw"/"scorecard"/"scorecard_FL_programs.csv"
df = pd.read_csv(target)
df = df.copy()

In [ ]:

print("BEFORE \n",df.columns.to_list())
before = df.copy()

def clean (df: pd.DataFrame) -> pd.DataFrame:

    #----------------------------------------- Standardizing column names
    df.columns = df.columns.str.replace(".", "_")
    df.columns = (
        df.columns
            .str.replace(r"(earnings_|overall_|staff_grad_plus_all_eval_inst_|not_enrolled_)","",regex=True)
    )
    df.columns = df.columns.str.strip()

    #----------------------------------------- Enforcing schema
    df["code"]=df["code"].astype(str)
    df["unit_id"]=df["unit_id"].astype(str)

    #----------------------------------------- Trimming + normalizing text fields
    df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
    df = df.apply(lambda col: col.str.replace("  "," ") if col.dtype == "object" else col) #replaces double spaces

    #----------------------------------------- Handling missing and sentinel values
    df = df.apply(lambda col: col.fillna(pd.NA) if col.dtype == "object" else col) #replaces empty cells with null
    df = df.apply(lambda col: col.str.replace(r"(NA|N/A|na|Na|n/a|nan|NAN|Nan|Null|NULL)",pd.NA, regex = True) if col.dtype == "object" else col) #replaces sentinel values with null
    numeric_cols = df.select_dtypes(include="number").columns
    df[numeric_cols] = df[numeric_cols].mask(df[numeric_cols] < 0, np.nan)

    #----------------------------------------- De-duplicating rows
    if df.duplicated().sum():
        print(f"{df.duplicated().sum()} duplicated rows found.")
        df = df.drop_duplicates()

    return df

print("AFTER \n",df.columns.tolist())
print("\nCleaned rows:\n",(before != df).sum().sum())
print(before.compare(df))


BEFORE 
 ['code', 'title', 'unit_id', 'distance', 'school.type', 'school.main_campus', 'credential.level', 'earnings.4_yr.overall_median_earnings', 'earnings.4_yr.working_not_enrolled.overall_count', 'debt.staff_grad_plus.all.eval_inst.median', 'debt.staff_grad_plus.all.eval_inst.median_payment', 'school.name']
AFTER 
 ['code', 'title', 'unit_id', 'distance', 'school.type', 'school.main_campus', 'credential.level', 'earnings.4_yr.overall_median_earnings', 'earnings.4_yr.working_not_enrolled.overall_count', 'debt.staff_grad_plus.all.eval_inst.median', 'debt.staff_grad_plus.all.eval_inst.median_payment', 'school.name']

Cleaned rows:
 0
Empty DataFrame
Columns: []
Index: []


In [ ]:
print(df.columns.to_list())
df.columns = df.columns.str.strip()
print(df.columns.to_list())

['code', 'title', 'unit_id', 'distance', 'school_type', 'school_main_campus', 'credential_level', '4_yr_median_earnings', '4_yr_working_count', 'debt_median', 'debt_median_payment', 'school_name']
['code', 'title', 'unit_id', 'distance', 'school_type', 'school_main_campus', 'credential_level', '4_yr_median_earnings', '4_yr_working_count', 'debt_median', 'debt_median_payment', 'school_name']


In [ ]:
df.info

<bound method DataFrame.info of       code                                              title  unit_id  \
0     3906               Theological and Ministerial Studies.   132408   
1     1101        Computer and Information Sciences, General.   132471   
2     1304        Educational Administration and Supervision.   132471   
3     2201                                               Law.   132471   
4     2401  Liberal Arts and Sciences, General Studies and...   132471   
...    ...                                                ...      ...   
1533  5202  Business Administration, Management and Operat...   496973   
1534  5202  Business Administration, Management and Operat...   496973   
1535  5202  Business Administration, Management and Operat...   496973   
1536  5138  Registered Nursing, Nursing Administration, Nu...   496991   
1537  1204  Cosmetology and Related Personal Grooming Serv...   497222   

      distance          school_type  school_main_campus  credential_level  \
0 

In [ ]:
df["code"]=df["code"].astype(str)
df["unit_id"]=df["unit_id"].astype(str)
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1538 entries, 0 to 1537
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   code                  1538 non-null   str    
 1   title                 1538 non-null   str    
 2   unit_id               1538 non-null   str    
 3   distance              1538 non-null   int64  
 4   school_type           1538 non-null   str    
 5   school_main_campus    1538 non-null   int64  
 6   credential_level      1538 non-null   int64  
 7   4_yr_median_earnings  1538 non-null   float64
 8   4_yr_working_count    1538 non-null   float64
 9   debt_median           1538 non-null   float64
 10  debt_median_payment   1538 non-null   float64
 11  school_name           1538 non-null   str    
dtypes: float64(4), int64(3), str(5)
memory usage: 144.3 KB


In [ ]:
before = df.copy()

df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
df = df.apply(lambda col: col.str.replace("  "," ") if col.dtype == "object" else col) #replaces double spaces
df = df.apply(lambda col: col.fillna(pd.NA) if col.dtype == "object" else col) #replaces empty cells with null
df = df.apply(lambda col: col.str.replace(r"(NA|N/A|na|Na|n/a|nan|NAN|Nan|Null|NULL)",pd.NA, regex = True) if col.dtype == "object" else col) #replaces sentinel values with null

print(before.compare(df))
print((before != df).sum().sum())

Empty DataFrame
Columns: []
Index: []
0


In [ ]:
import numpy as np

before = df.copy()

numeric_cols = df.select_dtypes(include="number").columns
df[numeric_cols] = df[numeric_cols].mask(df[numeric_cols] < 0, np.nan)

print(before.compare(df))
print((before != df).sum().sum())

Empty DataFrame
Columns: []
Index: []
0


In [ ]:
before = df.copy()

duplicate = df.duplicated
df = df.drop_duplicates()
if df.duplicated.sum():
    print(f"{duplicate.sum()} duplicated rows found.")

print(before.compare(df))
print((before != df).sum().sum())

Empty DataFrame
Columns: []
Index: []
0


In [ ]:
df.head()

,code,title,unit_id,distance,school_type,school_main_campus,credential_level,4_yr_median_earnings,4_yr_working_count,debt_median,debt_median_payment,school_name
0,3906,Theological and Ministerial Studies.,132408,3,"Private, nonprofit",1,3,31484.0,36.0,23946.0,254.0,Baptist University of Florida
1,1101,"Computer and Information Sciences, General.",132471,2,"Private, nonprofit",1,3,52714.0,66.0,27750.0,294.0,Barry University
2,1304,Educational Administration and Supervision.,132471,2,"Private, nonprofit",1,5,69477.0,87.0,51250.0,583.0,Barry University
3,2201,Law.,132471,1,"Private, nonprofit",1,7,62176.0,197.0,154477.0,1757.0,Barry University
4,2401,"Liberal Arts and Sciences, General Studies and...",132471,1,"Private, nonprofit",1,3,40411.0,36.0,38125.0,404.0,Barry University


In [30]:
def validate_ranges(df):
    rules = [
        ("credential_level", 1, 8),
        ("distance", 0, 3),
        ("school_main_campus", 0, 1),
        ("4_yr_median_earnings", 0, np.inf),
        ("4_yr_working_count", 0, np.inf),
        ("debt_median", 0, np.inf()),
        ("debt_median_payment", 0, np.inf())
        
    ]

    for col, low, high in rules:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")


    for col, low, high in rules:
        invalid = df.loc[~df[col].between(low, high)]
        
        if not invalid.empty:
            raise ValueError(
                f"{col} out of range [{low}, {high}]: "
                f"{invalid[col].unique()}"
            )